# 对ctree文件夹的一些分析

## 代码分析环节

（这个section中请大家撰写对代码的分析，其中的关键点）

1. ctree中主要的文件为cminimax.cpp和cnode.cpp，前者主要用于EfficientZero paper的Backup过程中的公式10，也就是分别协助计算$\min_{(s,a)}Q(s,a)$和$\max_{(s,a)}Q(s,a)$，也就是一些statistic的信息。
1. cminimax.cpp中的`normalize`函数实现了EfficientZero paper中公式10的过程，注意$\epsilon$为0.01，`delta`小于该值的话， 则直接用$\epsilon$。
1. cnode.cpp涵盖了MCTS中的流程中Selection 、Expansion 、Backup。
    1. 从Selection开始：对应代码，可以先观察`cselect_child`函数。如下：
        1. `cucb_score`计算得到了paper中公式7对应的结果（在arg max之前）。但是它维护一个了`max_index_lst`，其特点就是只要是大于等于`max_score - epsilon`都会被加入该list，并在函数的结尾处从该list中随机sample一个action返回。
        1. 考虑`cucb_score`如何计算，见函数`cucb_score`：大的结构为`prior_score + value_score`，其中`value_score`为公式7中$Q(s,a)$项，`prior_score`为剩余的部分。`value_score`的计算较为复杂，其需要考虑到`child->visit_count`为0的情况。`parent_mean_q`对应于公式8的$\hat{Q}(s)$。如果$N(s,a)$不为0，那么，就是一般的$Q(s,a)$，计算方式也就是`true_reward + discount * child->value()`。
        1. 不管何种方式，$Q(s,a)$都需要被normalize，也就是公式10中的做法，相比MuZero，其多了0.01的`delta`的下限。值得注意的是，代码上对`value_score`做了clip到$[0,1]$的操作。
    1. 回到MCTS的Selection部分。因为这里实现的是所谓的batch MCTS，所以，真正的Selection部分为函数`cbatch_traverse`。如下对该函数进行简单的解释：
        1. batch MCTS涉及到`results.num`棵树。在for循环中有所体现。
        1. 对每棵树，从root开始，所以while循环之前`is_root`为1，之后就为0，在while中进行了设置。
        1. while循环的前提为`node->expanded()`，意思就是`node`还不是leaf。`mean_q`的含义为公式8中的$\hat{Q}(s)$，在cucb_score函数中会有用（`child->visit_count == 0`时）。
        1. while循环所找到的叶子为`node`，其的`parent`在`results.search_paths[i]`中下标为`results.search_paths[i].size() - 2`。
        1. `hidden_state_index_x/y_lst`存放的均为`int`类型数据，即下标值。
    1. 如下进入Expansion环节：
        1. 涉及到的函数有`prepare`，及`prepare_no_noise`，分别对应公式9中$\rho>0$或$\rho=0$的情况。注意$\rho=0$是有用的，见paper中所阐述的：However, we do not use any noise and set $\rho$ to 0 instead for those **non-root** node or during evaluations.
        1. 考虑较为复杂的`prepare`函数：
            1. 因为是batch MCTS，所以，for循环涉及`this->root_num`棵树。
            1. `expand`函数在对leaf进行展开。该函数的主要环节为一种数值稳定的计算softmax的方法。
                1. $$\mathrm{softmax}(\mathbf{x})_i=\frac{\exp (x_i)}{\sum_j \exp (x_j)}$$
                1. $\mathbf{x}$更换为$\mathbf{z}=\mathbf{x}-\max_i x_i$
            1. `add_exploration_noise`就是公式9，即$(1-\rho)P(s,a)+\rho\mathcal{N}_\mathcal{D}(\xi)$，对于各个`child`好像可以采取不同的噪声，见`noises[a]`
    1. 最后考虑Backup部分：
        1. 该模块的起始点为`cbatch_back_propagate`函数，其考虑各个`results.nodes[i]`，从这个可以了解到`cbatch_traverse`函数中的最后一条语句`results.nodes.push_back(node);`，也就是说`cbatch_back_propagate`函数会先对各个leaf进行expand，然后，再从leaf到root进行backup。
        1. 实际实现backup功能的函数为`cback_propagate`。如下进行分析：
            1. for循环从下标`path_len - 1`开始考虑，该index对应leaf
            1. 分别对`node->value_sum`及`node->visit_count`进行更新。这个分别对应MuZero paper中的公式4。
            1. 接着获得`parent_value_prefix`，但如果是root，则该变量维持初始值0。否则考虑index `i - 1`，获得`parent->value_prefix`
            1. `bootstrap_value = true_reward + discount * bootstrap_value;`的理解如下：
                1. 其计算的为MuZero paper中的公式3，即$G^k$。但$G^k$和$G^{k-1}$的计算是有关的。
                1. $$G^k=\sum_{\tau=0}^{l-1-k}\gamma^\tau r_{k+1+\tau}+\gamma^{l-k}v^l$$
                1. $$
                \begin{split}
                G^{k-1}&=\sum_{\tau=0}^{l-1-(k-1)}\gamma^\tau r_{(k-1)+1+\tau}+\gamma^{l-(k-1)}v^l\\
                &=\sum_{0\le\tau\le l-1-(k-1)}\gamma^\tau r_{(k-1)+1+\tau}+\gamma\cdot\gamma^{l-k}v^l\\
                &=\sum_{0\le\tau+1\le l-1-(k-1)}\gamma^{\tau+1} r_{(k-1)+1+(\tau+1)}+\gamma\cdot\gamma^{l-k}v^l
                \\
                &=\sum_{-1\le\tau\le l-1-k}\gamma^{\tau+1} r_{k+1+\tau}+\gamma\cdot\gamma^{l-k}v^l\\
                &=r_k+\sum_{0\le\tau\le l-1-k}\gamma^{\tau+1} r_{k+1+\tau}+\gamma\cdot\gamma^{l-k}v^l\\
                &=r_k+\gamma G^k
                \end{split}
                $$
            1. 函数的最后确定`root`，以及对$\min_{(s,a)}Q(s,a)$和$\max_{(s,a)}Q(s,a)$等statistic进行清空，分别重置为`FLOAT_MAX`和`FLOAT_MIN`
            1. 进入到`update_tree_q`函数的调用，此函数分析如下：
                1. `update_tree_q`为基于stack的、非递归的depth-first search / DFS模块
                1. 惟一对MCTS有影响的部分在于`min_max_stats.update(qsa);`，其他的语句都是在实现DFS
    1. miscellaneous
        1. `get_trajectories`、`get_distributions`针对的是batch MCTS，可以直接考虑真正发挥作用的函数`get_trajectory`及`get_children_distribution`
        1. `get_trajectory`获取从root开始，沿`best_action`向下走。对于`node->best_action`来说，其的初始值为-1，且只在函数`cbatch_traverse`被改变，见`node->best_action = action;`，其中`action`由`cselect_child`返回，即EfficientZero中的公式7中的$\mathop{\arg\,\max}_a$
        1. `get_children_distribution`主要用于获得MCTS的策略，即EfficientZero中的公式11，即$$\pi(s,a)=\frac{N(s,a)^{1/T}}{\sum_b N(s,b)^{1/T}}$$
    
## 讨论环节

（这个section中，请大家提出对repo中难理解部分的疑问，其他同学或老师看到了会进行解答）

1. this is an example question. posted by lezhang.thu@gmail.com / cnode.cpp文件中，出现了大量的`is_reset`，不知道其语义是什么。
1. `expand`函数中，`ptr_node_pool->push_back(CNode(prior, action_num, ptr_node_pool));`，不知道为什么。`ptr_node_pool`又传入`CNode`的构造函数中，但`ptr_node_pool`在for循环中一直在更新。怀疑其和当前node获取children的列表有关，或许是当前node的children的list，怀疑。见函数`get_child`
1. `value`函数，本以为计算的是为MuZero paper中的公式4，但从代码中反复出现的语句`float qsa = true_reward + discount * node/child->value();`可以感觉到，$Q(s,a)$应该不是`value`函数的直接返回值。这一部分的逻辑没有理清楚。